# Congress House Bills Web Scraper

## Description

A web scraper for Congress House bills (HBNXXXXX)

## Imports and Dependencies

### Install Dependencies

In [1]:
!uv pip install -r requirements.txt

Audited 72 packages in 79ms


### Library Imports

In [2]:
from patchright.async_api import async_playwright, Page, Browser
from camoufox.async_api import AsyncCamoufox
import os
import json
import requests
from dotenv import load_dotenv

### Environment Variables

In [3]:
AWS_BUCKET_DATA_LOCATION = os.getenv("AWS_BUCKET_DATA_LOCATION")
AWS_BUCKET_METADATA_LOCATION = os.getenv("AWS_BUCKET_METADATA_LOCATION")


## Helper and Functions and Class Definitions

### File Class Object Definition

In [4]:
class File:
    def __init__(
            self,
            hbn : str,
            main_title : str, 
            session_number : str, 
            significance : str, 
            date_filed : str, 
            principal_authors : str, 
            date_read : str, 
            primary_referral : str, 
            bill_status : str,  
            text_filed : str, 
            is_file_downloadable : str
            ):
        self.hbn = hbn
        self.main_title = main_title
        self.session_number = session_number
        self.significance = significance
        self.date_filed = date_filed
        self.principal_authors = principal_authors
        self.date_read = date_read
        self.primary_referral = primary_referral
        self.bill_status = bill_status
        self.text_filed = text_filed
        self.is_file_downloadable = is_file_downloadable

    def __eq__(self, other):
        if isinstance(other, File):
            return self.hbn == other.hbn
        return False
    
    def __hash__(self):
        return hash(self.hbn)
files : set[File] = set()

### Global Variables

In [5]:
files : set[File] = set()
changes_counter = 0

### JSON Encoder and Progress Loading for File Object

In [6]:
def json_encoder(obj: File):
    """
    Encodes FIle class instance to JSON instance 

    Args:
        obj (File): File object

    Raises:
        TypeError: Occurs when object passed is not an instance of the File class

    Returns:
        dict[str,str]: dictionary for JSON parsing
    """
    if isinstance(obj, File):
        return {
            'House Bill Number' : obj.hbn,
            'Main Title' : obj.main_title,
            'Session Number' : obj.session_number,
            'Significance' : obj.significance,
            'Date Filed' : obj.date_filed,
            'Principal Authors' : obj.principal_authors,
            'Date Read' : obj.date_read,
            'Primary Referral' : obj.primary_referral,
            'Bill Status' : obj.bill_status,
            'Text Filed' : obj.text_filed
        }
    raise TypeError("Object is not JSON parsable.")

def load_files_from_json(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
            for item in data:
                # Reconstruct the File object using the JSON keys
                # We use .get() to avoid errors if a key is missing
                new_file = File(
                    hbn=item.get("House Bill Number"),
                    main_title=item.get("Main Title"),
                    session_number=item.get("Session Number"),
                    significance=item.get("Significance"),
                    date_filed=item.get("Date Filed"),
                    principal_authors=item.get("Principal Authors"),
                    date_read=item.get("Date Read"),
                    primary_referral=item.get("Primary Referral"),
                    bill_status=item.get("Bill Status"),
                    text_filed=item.get("Text Filed"),
                    is_file_downloadable=item.get("Downloaded", False)
                )
                files.add(new_file)
        print(f"Successfully loaded {len(files)} unique bills.")
    except FileNotFoundError:
        print("No existing JSON found. Starting with an empty set.")

### Download File From URL Function

In [7]:
def download(url: str, dest_folder: str):
    """
    Downloads the file from the URL provided and places it in the destination folder provided

    Inputs:
    url (str): input URL of file
    dest_folder (str): destination folder/directory of downloaded file
    
    Outputs:
    Returns 1 if the download was successful, and 0 if not.
    """
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist
    try:
        filename = url.split('/')[-1].replace(" ", "_")  
        file_path = os.path.join(dest_folder, filename)
        # print(f"URL: {url}")
        r = requests.get(url, stream=True)
        if r.ok:
            with open(file_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024 * 8):
                    if chunk:
                        f.write(chunk)
                        f.flush()
                        os.fsync(f.fileno())
        else:  
            return False
        return True
    except:
        return False

### Get Files from Current Page

In [8]:
async def get_files_from_page(hb_items_locator):
    """
    Gets all house bill files from the current page

    Args:
        hb_items_locator (list[Locator]): list of scraped house bill divs

    Returns:
        None
    """
    global changes_counter    
    count = await hb_items_locator.count()
    for i in range(count):
        hb_item = hb_items_locator.nth(i)
        
        # Trigger AOS animation
        await hb_item.scroll_into_view_if_needed()

        # Get Header Info
        hbn = await hb_item.locator("span.rounded.border span span").first.inner_text()
        main_title = await hb_item.locator("span.text-blue-500").first.inner_text()

        # Metadata Retrieval
        async def get_meta(label):
            try:
                # Logic: Find the label div, then get the very next div sibling (+)
                # only if it is inside the grid container
                value_locator = hb_item.locator(".grid.gap-1.px-5") \
                                       .locator(f"div:has-text('{label}') + div")
                
                text = await value_locator.first.inner_text(timeout=1000)
                return text.strip()
            except:
                return "N/A"

        # Check if file already exists in metadata
        is_hb_in_metadata = False
        matched_hb = next((u for u in files if u.hbn == hbn), None)
        if matched_hb:
            # print(f"{hbn}: Already in database. Checking for changes...")
            is_hb_in_metadata = True        

        # PDF Link
        pdf_loc = hb_item.locator('a[href$=".pdf"]').first
        link = await pdf_loc.get_attribute('href') if await pdf_loc.count() > 0 else "N/A"
        downloadability = False
        if link != 'N/A' and not is_hb_in_metadata:
            print(f"Downloading file: {hbn}...")
            downloadability = download(link, "outputs/")

        # Build File Object
        new_file = File(
            hbn.strip(),
            main_title.strip(),
            await get_meta("Session No. :"),
            await get_meta("Significance :"),
            await get_meta("Date Filed :"),
            await get_meta("Principal Author/s :"),
            await get_meta("Date Read :"),
            await get_meta("Primary Referral :"),
            await get_meta("Bill Status :"),
            link,
            downloadability # Downloadable
        )

        # Check for metadata changes if the file already exists
        if is_hb_in_metadata:
            # Check for changes with safeguard for N/A change due to possible bugs
            if new_file.bill_status != matched_hb.bill_status and new_file.bill_status != 'N/A':
                print(f"Change in Bill Status found. Updating bill status for {hbn}...")
                print(f"[LOG]\n   Old Bill Status: {matched_hb.bill_status}\n   New Bill Status: {new_file.bill_status}")
                matched_hb.bill_status = new_file.bill_status
                changes_counter += 1
                
        else:
            files.add(new_file)

## Scraper Stuff

### Actual Scraper

#### File Metadata Reset

In [ ]:
# Get updated metadata file from AWS bucket
print(AWS_BUCKET_METADATA_LOCATION)
!rm -rf outputs/*.pdf
!aws s3 cp {AWS_BUCKET_METADATA_LOCATION}metadata.json outputs  

# Populate files array for cross-checking
load_files_from_json('outputs/metadata.json')
changes_counter = 0


s3://aaia-raw/congress/20th_congress/metadata/
download: s3://aaia-raw/congress/20th_congress/metadata/metadata.json to outputs/metadata.json
Successfully loaded 8911 unique bills.


#### Scraper Code

In [ ]:

try:
    async with AsyncCamoufox(headless=False, geoip=True) as browser:
        context = await browser.new_context(viewport={"width":1000, "height":500})
        page = await context.new_page()

        await page.goto("https://congress.gov.ph/legislative-documents/")
        
        # Wait for initial load
        await page.wait_for_selector('[id="20th Congress"]', state='visible', timeout=90000)
        
        # Set pagination to 100
        await page.locator("select.form-select").nth(1).select_option('100')    
        
        # Open dropdown
        await page.locator('[id="20th Congress"]').click()
        
        # Initial scroll and wait for first page items
        await page.wait_for_selector('.cursor-pointer.rounded-sm.border', state='visible')
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")

        # Get last page number
        idx = 1
        last_page = int(await page.locator('ul.pagination > li').nth(-2).inner_text()) 
        
        while(idx <= last_page):
            # Scrape current page
            hb_items_locator = page.locator('.cursor-pointer.rounded-sm.border')
            await get_files_from_page(hb_items_locator)
            
            print(f"Finished scraping page {idx}")

            # Get "Next" button locator
            next_button = page.locator('li.next:not(.disabled) a') 
            
            if await next_button.count() > 0:
                # Capture ID of the first item to track when the data actually changes
                old_bill_id = await page.locator(".cursor-pointer span.rounded.border span span").first.inner_text()
                
                # Click next button
                await next_button.click()

                # 4. Wait for Content Refresh from Page Change
                try:
                    await page.wait_for_function(
                        f"""() => {{
                            const el = document.querySelector(".cursor-pointer span.rounded.border span span");
                            return el && el.innerText.trim() !== "{old_bill_id.strip()}";
                        }}""",
                        timeout=15000 # timeout buffer
                    )
                except:
                    # Fallback if JS check fails: wait for network to settle
                    await page.wait_for_load_state("networkidle")
                
                # Small buffer for the UI to stabilize
                await page.wait_for_timeout(3000)
                idx += 1
            else:
                print("No more pages available.")
                break
except Exception as e:
    print("Error occurred. Saving progress...")
    print(f"{e}")

# Processing logic (e.g., saving to JSON)
print(f"Number of Changes in Bills: {changes_counter}\n")
with open('outputs/metadata.json', mode='w', encoding='utf-8') as f:
    json.dump(
        obj=list(files),
        fp=f,
        default=json_encoder,
        indent=4
    )

90
Error occurred. Saving progress...
name 'idx' is not defined
Number of Changes in Bills: 0



### Upload to AWS Bucket

In [13]:
# Data Sync to S3 Bucket
!aws s3 sync outputs/ {AWS_BUCKET_DATA_LOCATION} --exclude "*" --include "*.pdf"

# Metadata upload to S3
!aws s3 cp outputs/metadata.json {AWS_BUCKET_METADATA_LOCATION}  
print(AWS_BUCKET_METADATA_LOCATION)


upload: outputs/CR00117.pdf to s3://aaia-raw/congress/20th_congress/data/CR00117.pdf
upload: outputs/CR00120.pdf to s3://aaia-raw/congress/20th_congress/data/CR00120.pdf
upload: outputs/CR00119.pdf to s3://aaia-raw/congress/20th_congress/data/CR00119.pdf
upload: outputs/HB07740.pdf to s3://aaia-raw/congress/20th_congress/data/HB07740.pdf 
upload: outputs/HB07743.pdf to s3://aaia-raw/congress/20th_congress/data/HB07743.pdf  
upload: outputs/HB07745.pdf to s3://aaia-raw/congress/20th_congress/data/HB07745.pdf
upload: outputs/HB07742.pdf to s3://aaia-raw/congress/20th_congress/data/HB07742.pdf
upload: outputs/HB07751.pdf to s3://aaia-raw/congress/20th_congress/data/HB07751.pdf
upload: outputs/HB07753.pdf to s3://aaia-raw/congress/20th_congress/data/HB07753.pdf
upload: outputs/HB07754.pdf to s3://aaia-raw/congress/20th_congress/data/HB07754.pdf
upload: outputs/HB07752.pdf to s3://aaia-raw/congress/20th_congress/data/HB07752.pdf
upload: outputs/HB07755.pdf to s3://aaia-raw/congress/20th_con